In [ ]:
%load_ext dotenv
%dotenv ../redbox/.env.test

In [ ]:
from langgraph.prebuilt import create_react_agent
from redbox.models.chain import ChatLLMBackend, RedboxQuery, RedboxState
from redbox.loader.loaders import get_chat_llm
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from redbox.models.settings import Settings

def get_state(user_uuid, prompts, documents, ai_setting, permitted_s3_keys):
    q = RedboxQuery(
        question=prompts,
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=[],
        ai_settings=ai_setting,
        permitted_s3_keys=permitted_s3_keys,
    )

    return RedboxState(
        request=q,
    )
env = Settings()
model = get_chat_llm(ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))

Using Caddy MCP as a tool in react agent

In [ ]:
async with MultiServerMCPClient(
    {
        "caddy": {
            "url": env.mcp_caddy_url,
            "transport": "sse",
            "headers": {env.mcp_headers: env.mcp_caddy_token}
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    result = await agent.ainvoke({"messages": "Search for grants relating to badgers."})

Using Parlex MCP as a tool in react agent

In [ ]:
async with MultiServerMCPClient(
    {
        "parlex": {
            "url": env.mcp_parlex_url,
            "transport": "sse",
            "headers": {env.mcp_headers: env.mcp_parlex_token}
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    result = await agent.ainvoke({"messages": "Search for Keir Starmer"})